In [1]:
import numpy as np
# Load pre-existing dataset Iris dataset as mentioned in the book
from sklearn.datasets import load_iris
dataset = load_iris()
X = dataset.data               #set of continuous feature values
y = dataset.target             #set of target classes
#print(dataset.DESCR)
n_samples, n_features = X.shape # this gives dimension of loaded feature dataset

In [2]:
#converting continous values to categorical values
#logic used - compute the mean and determine whether a value is above or below the mean.

# Compute the mean for each attribute
attribute_means = X.mean(axis=0)                   #Column-wise / feature-wise mean
assert attribute_means.shape == (n_features,)      #check if the means count is equal to total feature number
X_d = np.array(X >= attribute_means, dtype='int')  #finally,changing it to categorical feature values

In [3]:
#we need to use imported data for training as well as splitting
#so, this can be achieved by using sklearn.model_selection

from sklearn.model_selection import train_test_split

# Set the random state to the same number to get the same results as in the book
random_state = 14

X_train, X_test, y_train, y_test = train_test_split(X_d, y, random_state=random_state)
print("There are {} training samples".format(y_train.shape))
print("There are {} testing samples".format(y_test.shape))

There are (112,) training samples
There are (38,) testing samples


In [4]:
#define 2 functions - 1. to Compute the predictors and error for a given feature using the OneR algorithm
#                     2. to count how frequency  give certain predictions

from collections import defaultdict
from operator import itemgetter

#################### first function ##################

def train(X, y_true, feature): 
    #X is a dataset , y_true - corresponding class values , feature - current feature which is being modeled
    # Check that variable is a valid number
    n_samples, n_features = X.shape
    assert 0 <= feature < n_features
    
    # Get all of the unique values that this variable has using set function
    values = set(X[:,feature])
    
    # Stores the predictors array that is returned
    predictors = dict()
    errors = []
    
    for current_value in values:  #loop through all the unique values of current feature
        most_frequent_class, error = train_feature_value(X, y_true, feature, current_value)
        predictors[current_value] = most_frequent_class
        errors.append(error)
        
    # Compute the total error of using this feature to classify on
    total_error = sum(errors)
    return predictors, total_error

In [5]:
#################### second function ##################

def train_feature_value(X, y_true, feature, value):
    
    # Create a simple dictionary to count how frequency they give certain predictions
    class_counts = defaultdict(int)
    
    # Iterate through each sample and count the frequency of each class/value pair
    for sample, y in zip(X, y_true):
        if sample[feature] == value:
            class_counts[y] += 1
            
    # Now get the best one by sorting (highest first) and choosing the first item
    sorted_class_counts = sorted(class_counts.items(), key=itemgetter(1), reverse=True)
    most_frequent_class = sorted_class_counts[0][0]
    
    # The error is the number of samples that do not classify as the most frequent class
    # *and* have the feature value.
    n_samples = X.shape[1]
    error = sum([class_count for class_value, class_count in class_counts.items()
                 if class_value != most_frequent_class])
    
    return most_frequent_class, error

In [6]:
# Compute all of the predictors using functions defined in previous steps

all_predictors = {variable: train(X_train, y_train, variable) for variable in range(X_train.shape[1])}

print(all_predictors)
print()

errors = {variable: error for variable, (mapping, error) in all_predictors.items()}

# Now choose the best and save that as "model"
# Sort by error
best_variable, best_error = sorted(errors.items(), key=itemgetter(1))[0]

print("The best model is based on variable {0} and has error {1:.2f}".format(best_variable, best_error))

# Choose the bset model
model = {'variable': best_variable,
         'predictor': all_predictors[best_variable][0]}
print(model)

{0: ({0: 0, 1: 2}, 41), 1: ({0: 1, 1: 0}, 58), 2: ({0: 0, 1: 2}, 37), 3: ({0: 0, 1: 2}, 37)}

The best model is based on variable 2 and has error 37.00
{'variable': 2, 'predictor': {0: 0, 1: 2}}


In [7]:
def predict(X_test, model):
    variable = model['variable']
    predictor = model['predictor']
    y_predicted = np.array([predictor[int(sample[variable])] for sample in X_test])
    return y_predicted

In [8]:
y_predicted = predict(X_test, model)
print("Predicted classes")
print(y_predicted)
print("Test Classes")
print(y_test)

Predicted classes
[0 0 0 2 2 2 0 2 0 2 2 0 2 2 0 2 0 2 2 2 0 0 0 2 0 2 0 2 2 0 0 0 2 0 2 0 2
 2]
Test Classes
[0 0 0 1 2 1 0 1 0 1 2 0 2 2 0 1 0 2 2 1 0 0 0 1 0 2 0 1 1 0 0 1 1 0 1 0 2
 1]


In [9]:
# Compute the accuracy by taking the mean of the amounts that y_predicted is equal to y_test
accuracy = np.mean(y_predicted == y_test) * 100
print("The test accuracy is {:.1f}%".format(accuracy))

The test accuracy is 65.8%


In [10]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97        17
           1       0.00      0.00      0.00        13
           2       0.40      1.00      0.57         8

   micro avg       0.66      0.66      0.66        38
   macro avg       0.45      0.67      0.51        38
weighted avg       0.51      0.66      0.55        38



c:\users\sheth\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
